In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/ProjC/Nov11_newModel/competition-project-2

 attempt-1.ipynb   experiments	 Plots.docx			  test_data.zip
 data.zip	   ml_utils	'Ratna Copy of attempt-1.ipynb'


In [ ]:
!cp /content/drive/MyDrive/ProjC/Nov11_newModel/data.zip .

In [ ]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: __MACOSX/._data         
   creating: data/preprocessed_data/
  inflating: data/.DS_Store          
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/.gitignore         
  inflating: __MACOSX/data/._.gitignore  
   creating: data/splits/
   creating: data/splits/test/
   creating: data/splits/train/
   creating: data/splits/val/
  inflating: data/splits/test/subject_007_session_04__y.csv  
  inflating: data/splits/test/subject_007_session_04__x.csv  
  inflating: data/splits/test/subject_007_session_02__x.csv  
  inflating: data/splits/test/subject_007_session_02__y.csv  
  inflating: data/splits/test/subject_008_session_01__y.csv  
  inflating: data/splits/test/subject_006_session_02__x.csv  
  inflating: data/splits/test/subject_006_session_02__y.csv  
  inflating: data/splits/test/subject_008_session_01__x.csv  
  inflating: data/splits/test/subject_005_session_03__y.csv  
  inflating: data/splits/test/subject_005_session_0

In [ ]:
!rm data.zip

### Replace [YOUR_ACCESS_TOKEN] with your GitHub access token (remove the brackets)

In [ ]:
!git clone https://savadikarc:<ACCESSTOKEN>@github.com/savadikarc/ece542-competition-project.git

Cloning into 'ece542-competition-project'...
remote: Enumerating objects: 259, done.
remote: Counting objects: 100% (259/259), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 259 (delta 142), reused 176 (delta 72), pack-reused 0
Receiving objects: 100% (259/259), 279.86 KiB | 2.45 MiB/s, done.
Resolving deltas: 100% (142/142), done.


In [ ]:
!cd ece542-competition-project && git checkout phase-1 && git pull origin phase-1

Already on 'phase-1'
Your branch is up to date with 'origin/phase-1'.
From https://github.com/savadikarc/ece542-competition-project
 * branch            phase-1    -> FETCH_HEAD
Already up to date.


In [ ]:
base_path = "/content/drive/MyDrive/ProjC/Nov11_newModel/competition-project-2"

In [ ]:
import os
import sys
sys.path.append("ece542-competition-project")

In [ ]:
import json
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from torch.utils.data.sampler import WeightedRandomSampler
import numpy as np
from ml_utils.dataset import SubjectDataset

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cpu


In [ ]:
base_data_path = os.path.join("data", "splits")
train_data_path = os.path.join(base_data_path, "train")
val_data_path = os.path.join(base_data_path, "val")

splits_file = os.path.join(base_path, "experiments", "metadata", "split_ids.json")
with open(splits_file, "r") as f:
    split_ids = json.load(f)

In [ ]:
stats_path = os.path.join(base_path, "experiments", "metadata", "statistics.json")
with open(stats_path, "r") as f:
    stats = json.load(f)

In [ ]:
min = np.array([v["min"] for k, v in stats.items()])
max = np.array([v["max"] for k, v in stats.items()])

min = torch.from_numpy(min).float()
min = torch.unsqueeze((torch.unsqueeze(min, 0)), -1)
min = min.to(device)
max = torch.from_numpy(max).float().to(device)
max = torch.unsqueeze((torch.unsqueeze(max, 0)), -1)
max = max.to(device)

In [ ]:
min.shape

torch.Size([1, 6, 1])

In [ ]:
# Training hyperparameters
batch_size = 128
num_epochs = 30

In [ ]:
train_dataset = SubjectDataset(
    train_data_path, 
    split_ids["train"]
)
ys = train_dataset.y.tolist()
counts = Counter(ys)
weights = np.array([1./counts[_y] for _y in ys])
sample_weights = torch.from_numpy(weights).float()
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
train_iterations = (len(train_dataset) // batch_size) + ((len(train_dataset) % batch_size) != 0)

val_dataset = SubjectDataset(
    val_data_path, 
    split_ids["val"]
)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
val_iterations = (len(val_dataset) // batch_size) + ((len(val_dataset) % batch_size) != 0)

Converting uid 005_02
Converting uid 001_06
Converting uid 003_02
Converting uid 001_05
Converting uid 002_02
Converting uid 003_01
Converting uid 003_03
Converting uid 005_01
Converting uid 001_07
Converting uid 002_05
Converting uid 004_02
Converting uid 002_03
Converting uid 001_02
Converting uid 002_04
Converting uid 001_03
Converting uid 004_01
Converting uid 001_08
Converting uid 002_01
Converting uid 001_01
Converting uid 001_04


In [ ]:
print(train_iterations)
print(val_iterations)

1373
422


In [ ]:
class OneDConvNet(nn.Module):
  def __init__(self, n_features, n_classes, base_filters=32):
    super(OneDConvNet, self).__init__()

    self.conv1 = nn.Conv1d(in_channels=n_features, out_channels=base_filters, kernel_size=5, stride=1, padding=1)
    self.norm1 = nn.LayerNorm(base_filters*118)
    self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
    self.conv2 = nn.Conv1d(in_channels=base_filters, out_channels=base_filters*2, kernel_size=5, stride=1, padding=1)
    self.norm2 = nn.LayerNorm(base_filters*2*57)
    self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)
    self.conv3 = nn.Conv1d(in_channels=base_filters*2, out_channels=base_filters*4, kernel_size=5, stride=1, padding=1)
    self.norm3 = nn.LayerNorm(base_filters*4*26)
    self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)
    self.conv4 = nn.Conv1d(in_channels=base_filters*4, out_channels=base_filters*8, kernel_size=5, stride=1, padding=1)
    self.norm4 = nn.LayerNorm(base_filters*8*11)
    self.fc1 = nn.Linear(base_filters*8, base_filters*16)
    self.dropout5 = nn.Dropout(0.4)
    self.fc2 = nn.Linear(base_filters*16, n_classes)
  
  def forward(self, x):
    x = self.conv1(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm1(x)
    x = x.view(N, C, T)
    x = self.pool1(F.relu(x))

    x = self.conv2(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm2(x)
    x = x.view(N, C, T)
    x = self.pool2(F.relu(x))

    x = self.conv3(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm3(x)
    x = x.view(N, C, T)
    x = self.pool3(F.relu(x))

    x = self.conv4(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm4(x)
    x = x.view(N, C, T)
    x = F.relu(x)

    N, C, T = x.size()
    x = x.mean(dim=-1) # Flatten
    x = self.dropout5(F.relu(self.fc1(x)))
    x = self.fc2(x)
    return x

model = OneDConvNet(6, 4).to(device)

In [ ]:
# import numpy as np
# X = np.random.randn(1, 6, 40)
# T = torch.from_numpy(X).float().to(device)
# y = model(T)

In [ ]:
# y.shape

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# optimizer = optim.RMSprop(model.parameters(), lr=1e-3)
scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=1, verbose=True)
criterion = nn.CrossEntropyLoss()

In [ ]:
def train_step(X, y, model, optimizer, criterion):

    y_pred = model(X)
    predicted_classes = torch.argmax(y_pred.detach(), dim=1)

    loss = criterion(y_pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    corrects = torch.sum(y.data == predicted_classes)

    return loss.item(), corrects

def val_step(X, y, model, criterion):

    with torch.no_grad():
        
        y_pred = model(X)
        predicted_classes = torch.argmax(y_pred.detach(), dim=1)
        loss = criterion(y_pred, y)
        corrects = torch.sum(y.data == predicted_classes)

    return loss.item(), corrects, predicted_classes.detach().cpu().numpy()

In [ ]:
best_val_loss = float("inf")

for epoch in range(num_epochs):
    # Train for "n" number of iterations
    running_loss = 0.
    running_acc = 0.
    for iteration, (X, y) in enumerate(train_dataloader):

        X = X.float().to(device)
        # Normalize
        X = (X - min) / (max - min)

        y = y.view(X.size(0)).to(device)

        loss, corrects = train_step(X, y, model, optimizer, criterion)

        # Running metrics
        running_loss = running_loss + loss * X.size(0)
        running_acc = running_acc + corrects

        if iteration % 100 == 0:
            print(f"Iteration: {iteration}/{train_iterations} | train_loss: {loss} | train_acc: {corrects/X.size(0)}")

    train_loss = running_loss / len(train_dataset)
    train_acc = running_acc / len(train_dataset)

    # Validate
    running_val_loss = 0.
    running_val_acc = 0.
    for step, (X, y) in enumerate(val_dataloader):

        X = X.float().to(device)
        X = (X - min) / (max - min)

        y = y.view(X.size(0)).to(device)

        loss, corrects, predicted_classes = val_step(X, y, model, criterion)
        # Running metrics
        running_val_loss = running_val_loss + loss * X.size(0)
        running_val_acc = running_val_acc + corrects

    val_loss = running_val_loss / len(val_dataset)
    val_acc = running_val_acc / len(val_dataset)

    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        # Checkpoint model
        path = os.path.join(base_path, "experiments", "models", "checkpoint_instance_norm_dropout_sgd_lr3.pth")
        print(f"Saving model to {path}")
        torch.save(model.state_dict(), path)
        best_val_loss = val_loss

    print(f"Epoch: {epoch} | train_loss {train_loss} | train_acc: {train_acc} | val_loss: {val_loss} | val_acc: {val_acc}")

Iteration: 0/1373 | train_loss: 1.415602684020996 | train_acc: 0.2265625
Iteration: 100/1373 | train_loss: 1.3832768201828003 | train_acc: 0.3515625
Iteration: 200/1373 | train_loss: 1.3525331020355225 | train_acc: 0.359375
Iteration: 300/1373 | train_loss: 1.408145546913147 | train_acc: 0.234375
Iteration: 400/1373 | train_loss: 1.3620892763137817 | train_acc: 0.3125
Iteration: 500/1373 | train_loss: 1.217760443687439 | train_acc: 0.3828125
Iteration: 600/1373 | train_loss: 0.7912154197692871 | train_acc: 0.7109375
Iteration: 700/1373 | train_loss: 0.633650004863739 | train_acc: 0.7109375
Iteration: 800/1373 | train_loss: 0.37608540058135986 | train_acc: 0.7890625
Iteration: 900/1373 | train_loss: 0.32395899295806885 | train_acc: 0.875
Iteration: 1000/1373 | train_loss: 0.3931499421596527 | train_acc: 0.7890625
Iteration: 1100/1373 | train_loss: 0.33720874786376953 | train_acc: 0.8671875
Iteration: 1200/1373 | train_loss: 0.33655256032943726 | train_acc: 0.859375
Iteration: 1300/1373 

In [ ]:
np.array([train_dataset.y[0]])

array([0])

In [ ]:
best_model = OneDConvNet(6, 4)
best_model.load_state_dict(torch.load(os.path.join(base_path, "experiments", "models", "checkpoint_instance_norm_dropout_sgd_lr3.pth")))
best_model = best_model.to(device)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

class Evaluation:

  def __init__(self):
        pass
  
  def aggregate_metrics(self, y, y_hat):
    
    """ Metrics for entire set 
        
        Arguments passed are: 
                      y     : ground truth  
                      y_hat : predicted class
        Calculations:
                  accuracy  : (tp + tn) / (total samples =(p + n))
                  precision : tp / (tp + fp)
                  recall    : tp / (tp + fn)
                  f1        : 2 tp / (2 tp + fp + fn)
    """
    accuracy  = accuracy_score(y, y_hat)
    precision = precision_score(y, y_hat, average="macro")
    recall    = recall_score(y, y_hat, average="macro")
    f1        = f1_score(y, y_hat, average="macro")
    return {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1": f1
        }

  def classwise_metrics(self, y, y_hat):

    """ Metrics for each class. 
        Average is set to macro for calculating the score of each label, and find their unweighted mean. 
        This does not take label imbalance into account.
        
        Arguments passed are: 
                      y     : ground truth  
                      y_hat : predicted class
        Calculations:
                  accuracy  : (tp + tn) / (total samples =(p + n))
                  precision : tp / (tp + fp)
                  recall    : tp / (tp + fn)
                  f1        : 2 tp / (2 tp + fp + fn)
    """
    
    class_precision = precision_score(y, y_hat, average=None)
    class_recall    = recall_score(y, y_hat, average=None)
    class_f1        = f1_score(y, y_hat, average=None)
    
    return {
        "Precision": class_precision,
        "Recall": class_recall,
        "F1": class_f1
    }

In [ ]:
evaluator = Evaluation()

In [ ]:
val_dataset = SubjectDataset(
    val_data_path, 
    split_ids["val"]
)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
val_iterations = (len(val_dataset) // batch_size) + ((len(val_dataset) % batch_size) != 0)

Converting uid 001_08
Converting uid 002_01
Converting uid 001_01
Converting uid 001_04


In [ ]:
output = []
labels = []
for X, y in val_dataloader:
    X = X.float().to(device)
    X = (X - min) / (max - min)
    
    y = y.view(X.size(0)).to(device)

    y_pred = best_model(X)
    predicted_classes = torch.argmax(y_pred, dim=1).detach().cpu().numpy()
    y_true = y.cpu().numpy()

    output.append(predicted_classes)
    labels.append(y_true)

In [ ]:
_output = np.concatenate(output, axis=0)
_labels = np.concatenate(labels, axis=0)

In [ ]:
agg_metrics = evaluator.aggregate_metrics(_labels, _output)
print("Agg metrics")
print(agg_metrics)
classwise_metrics = evaluator.classwise_metrics(_labels, _output)
print("Classwise Metrics")
print(classwise_metrics)

Agg metrics
{'Accuracy': 0.7798638294714579, 'Precision': 0.7117290408103055, 'Recall': 0.7789391168841989, 'F1': 0.7258035050631524}
Classwise Metrics
{'Precision': array([0.81566077, 0.66864785, 0.59642284, 0.76618471]), 'Recall': array([0.8692247 , 0.9155646 , 0.85756677, 0.4734004 ]), 'F1': array([0.84159132, 0.77286389, 0.70354341, 0.5852154 ])}


In [ ]:
test_data_path = os.path.join(base_data_path, "test")
test_dataset = SubjectDataset(
    test_data_path, 
    split_ids["test"]
)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
test_iterations = (len(test_dataset) // batch_size) + ((len(test_dataset) % batch_size) != 0)

Converting uid 005_03
Converting uid 006_01
Converting uid 006_02
Converting uid 006_03
Converting uid 007_01
Converting uid 007_02
Converting uid 007_03
Converting uid 007_04
Converting uid 008_01


In [ ]:
output = []
labels = []
for X, y in test_dataloader:
    X = X.float().to(device)
    X = (X - min) / (max - min)
    
    y = y.view(X.size(0)).to(device)

    y_pred = best_model(X)
    predicted_classes = torch.argmax(y_pred, dim=1).detach().cpu().numpy()
    y_true = y.cpu().numpy()

    output.append(predicted_classes)
    labels.append(y_true)

In [ ]:
_output = np.concatenate(output, axis=0)
_labels = np.concatenate(labels, axis=0)

In [ ]:
agg_metrics = evaluator.aggregate_metrics(_labels, _output)
print("Agg metrics")
print(agg_metrics)
classwise_metrics = evaluator.classwise_metrics(_labels, _output)
print("Classwise Metrics")
print(classwise_metrics)

Agg metrics
{'Accuracy': 0.9183509481764384, 'Precision': 0.8324434549092692, 'Recall': 0.9222066561031865, 'F1': 0.872718448091402}
Classwise Metrics
{'Precision': array([0.96535883, 0.74620445, 0.83529412, 0.78291642]), 'Recall': array([0.93114962, 0.94914709, 0.96978477, 0.83874515]), 'F1': array([0.94794569, 0.83552911, 0.89752921, 0.80986978])}


In [ ]:
path = os.path.join(base_path, "experiments", "models", "checkpoint_instance_norm_dropout_sgd_lr3_long.pth")
print(f"Saving model to {path}")
torch.save(model.state_dict(), path)

Saving model to /content/drive/MyDrive/ProjC/Nov11_newModel/competition-project-2/experiments/models/checkpoint_instance_norm_dropout_sgd_lr3_long.pth
